# Protocols, records, and types

## The expression problem

In [ ]:
; A simple map containing expense information:

(ns clj-in-act.ch9.expense
    (:import [java.text SimpleDateFormat]))

(defn new-expense [date-string dollars cents category merchant-name]
    {:date (.parse (SimpleDateFormat. "yyyy-MM-dd") date-string)
     :amount-dollars dollars
     :amount-cents cents
     :category category
     :merchant-name merchant-name})


In [ ]:
; A function 'total-cents' that computes the expense amount in cents:

(defn total-cents [e]
    (-> (:amount-dollars e)
        (* 100)
        (+ (:amount-cents e))))


In [ ]:
; A function 'total-amount' to calculate the total amount, given a list of expenses, 
; and a criteria function by which to select expenses from a list:

(defn total-amount
    ([expenses-list]
        (total-amount (constantly true) expenses-list))
    ([pred expenses-list]
        (->> expenses-list
             (filter pred)
             (map total-cents)
             (apply +))))


In [ ]:
; Predicate functions that can be used with 'total-amount' 
; to select a particular category of expenses:

(defn is-category? [e some-category]
    (= (:category e) some-category))

(defn category-is [category]
    #(is-category? % category))


In [ ]:
; Sample data to test the above functions

(ns clj-in-act.ch9.expense-test
    (:require [clj-in-act.ch9.expense :refer :all]
              [clojure.test :refer :all]))

(def clj-expenses [(new-expense "2009-8-20" 21 95 "books" "amazon.com")
                   (new-expense "2009-8-21" 72 43 "food" "mollie-stones")
                   (new-expense "2009-8-22" 315 71 "car-rental" "avis")
                   (new-expense "2009-8-23" 15 68 "books" "borders")])


In [ ]:
; Test the last clojure functions

(deftest test-clj-expenses-total
    (is (= 42577 (total-amount clj-expenses)))
    (is (= 3763 (total-amount (category-is "books") clj-expenses))))

(run-tests 'clj-in-act.ch9.expense-test)

In [ ]:
; A java legacy Expense class

"""
package com.curry.expenses;
import java.util.Calendar;
import java.text.SimpleDateFormat;
import java.text.ParseException;


public class Expense {
    private Calendar date;
    private int amountDollars;
    private int amountCents;
    private String merchantName;
    private String category;

    public Expense(
        String dateString, int amountDollars, int amountCents,
        String category, String merchantName
    ) throws ParseException {
        this.date = Calendar.getInstance();
        this.date.setTime(
            new SimpleDateFormat(\"yyyy-MM-dd\")
                .parse(dateString)
         );
        this.amountDollars = amountDollars;
        this.amountCents = amountCents;
        this.merchantName = merchantName;
        this.category = category;
    }
    
    public Calendar getDate() {
        return date;
    }
    
    public int getAmountDollars() {
        return amountDollars;
    }

    public int getAmountCents() {
        return amountCents;
    }
    public String getMerchantName() {
        return merchantName;
    }
    public String getCategory() {
        return category;
    }
    public int amountInCents() {
        return this.amountDollars*100 + this.amountCents;
    }
}
"""


In [ ]:
; Test the last java class

(ns clj-in-act.ch9.expense-test
    (:require [clj-in-act.ch9.expense :refer :all]
              [Expense :refer :all]))

(def java-expenses [(Expense. "2009-8-24" 44 95 "books" "amazon.com")
                    (Expense. "2009-8-25" 29 11 "gas" "shell")])

(deftest test-java-expenses-total
    (let [total-cents (map #(.amountInCents %) java-expenses)]
    (is (= 7406 (apply + total-cents)))))


In [ ]:
; Test both the clojure functions and the java class
; (the test won't pass)

(def mixed-expenses 
    (concat clj-expenses java-expenses))

(deftest test-mixed-expenses-total
    (is (= 49983 (total-amount mixed-expenses)))
    (is (= 8258 (total-amount (category-is "books") mixed-expenses))))


In [ ]:
; To handle the new data type (the java Expense class),
; the 'total-cents' function is redefined as a multimethod

(defmulti total-cents class)
(defmethod total-cents clojure.lang.IPersistentMap [e] ; Handles Clojure data types
    (-> (:amount-dollars e)
        (* 100)
        (+ (:amount-cents e))))
(defmethod total-cents com.curry.expenses.Expense [e] ; Handles Java data types
    (.amountInCents e))


In [ ]:
; The 'is-category?' function is also redefined as a multimethod

(defmulti is-category? (fn [e category] (class e)))
(defmethod is-category? clojure.lang.IPersistentMap [e some-category] ; Handles clojure data types
    (= (:category e) some-category))
(defmethod is-category? com.curry.expenses.Expense [e some-category] ; Handles Java data types
    (= (.getCategory e) some-category))


## Examining the operations side of the expression problem

In [ ]:
; Helper functions for the new 'def-modus-operandi' construct

(defn dispatch-fn-for [method-args]
    `(fn ~method-args (class ~(first method-args))))

(defn expand-spec [[method-name method-args]]
    `(defmulti ~method-name ~(dispatch-fn-for method-args)))

(defmacro def-modus-operandi [mo-name & specs]
    `(do ~@(map expand-spec specs)))


In [ ]:
; Helper functions for the new 'detail-modus-operandi' construct

(defn expand-method [data-type [name & body]]
    `(defmethod ~name ~data-type ~@body))

(defmacro detail-modus-operandi [mo-name data-type & fns]
    `(do ~@(map #(expand-method data-type %) fns)))


In [ ]:
; A modified version of 'def-modus-operandi' that 
; stores the info about its methods, stored in a map

(defn mo-method-info [[name args]]
    {(keyword name) {:args `(quote ~args)}})

(defn mo-methods-registration [specs]
    (apply merge (map mo-method-info specs)))

(defmacro def-modus-operandi [mo-name & specs]
    `(do
        (def ~mo-name ~(mo-methods-registration specs))
        ~@(map expand-spec specs)))

(def-modus-operandi ExpenseCalculations
    (total-cents [e])
    (is-category? [e category]))
    
ExpenseCalculations

In [ ]:
; A modified version of 'detail-modus-operandi' that 
; modifies the info about its methods, stored in a map

(defn expand-method [mo-name data-type [method-name & body]]
    `(do
        (alter-var-root (var ~mo-name) update-in 
            [(keyword '~method-name) :implementors] 
            conj ~data-type)
        (defmethod ~method-name ~data-type ~@body)))

(defmacro detail-modus-operandi [mo-name data-type & fns] 
    `(do
        ~@(map #(expand-method mo-name data-type %) fns)))


In [ ]:
; First version of the app, implemented 
; with 'def-modus-operandi' and 'detail-modus-operandi' 

(ns clj-in-act.ch9.modus-operandi)

(defn dispatch-fn-for [method-args]
    `(fn ~method-args (class ~(first method-args))))

(defn expand-spec [[method-name method-args]]
    `(defmulti ~method-name ~(dispatch-fn-for method-args)))

(defn mo-method-info [[name args]]
    {(keyword name) {:args `(quote ~args)}})

(defn mo-methods-registration [specs]
    (apply merge (map mo-method-info specs)))

(defmacro def-modus-operandi [mo-name & specs]
    `(do
        (def ~mo-name ~(mo-methods-registration specs))
        ~@(map expand-spec specs)))

(defn expand-method [mo-name data-type [method-name & body]]
    `(do
    (alter-var-root (var ~mo-name) update-in 
        [(keyword '~method-name) :implementors] 
        conj ~data-type)
    (defmethod ~method-name ~data-type ~@body)))

(defmacro detail-modus-operandi [mo-name data-type & fns] 
    `(do
        ~@(map #(expand-method mo-name data-type %) fns)))


In [ ]:
; Adding support to clojure maps with 'detail-modus-operandi'

(detail-modus-operandi ExpenseCalculations
    clojure.lang.IPersistentMap
    (total-cents [e]
        (-> (:amount-dollars e)
            (* 100)
            (+ (:amount-cents e))))
    (is-category? [e some-category]
        (= (:category e) some-category)))


In [ ]:
; Adding support to the java Expense class with 'detail-modus-operandi'

(detail-modus-operandi ExpenseCalculations
    com.curry.expenses.Expense
    (total-cents [e]
        (.amountInCents e))
    (is-category? [e some-category]
        (= (.getCategory e) some-category)))


In [ ]:
; Second version of the app, implemented with 
; 'def-modus-operandi' and 'detail-modus-operandi'

(ns clj-in-act.ch9.expense-modus-operandi
    (:require [clj-in-act.ch9.modus-operandi :refer :all])
    (:import [java.text SimpleDateFormat]
             [java.util Calendar]))

(defn new-expense [date-string dollars cents category merchant-name]
    (let [calendar-date (Calendar/getInstance)]
        (.setTime calendar-date (.parse (SimpleDateFormat. "yyyy-MM-dd") date-string))
        {:date calendar-date
         :amount-dollars dollars
         :amount-cents cents
         :category category
         :merchant-name merchant-name}))

(def-modus-operandi ExpenseCalculations
    (total-cents [e])
    (is-category? [e category]))

(detail-modus-operandi ExpenseCalculations
    clojure.lang.IPersistentMap
    (total-cents [e]
        (-> (:amount-dollars e)
            (* 100)
            (+ (:amount-cents e))))
    (is-category? [e some-category]
        (= (:category e) some-category)))

(detail-modus-operandi ExpenseCalculations
    com.curry.expenses.Expense
    (total-cents [e]
        (.amountInCents e))
    (is-category? [e some-category]
        (= (.getCategory e) some-category)))

(defn category-is [category]
    #(is-category? % category))

(defn total-amount
    ([expenses-list]
        (total-amount (constantly true) expenses-list))
    ([pred expenses-list]
        (->> expenses-list
            (filter pred)
            (map total-cents)
            (apply +))))


In [ ]:
; Testing the implementation of modus operandi

(ns clj-in-act.ch9.expense-test
    (:import [com.curry.expenses Expense])
    (:require [clj-in-act.ch9.expense-modus-operandi :refer :all]
              [clojure.test :refer :all]))

(def clj-expenses [(new-expense "2009-8-20" 21 95 "books" "amazon.com")
                   (new-expense "2009-8-21" 72 43 "food" "mollie-stones")
                   (new-expense "2009-8-22" 315 71 "car-rental" "avis")
                   (new-expense "2009-8-23" 15 68 "books" "borders")])

(deftest test-clj-expenses-total
    (is (= 42577 (total-amount clj-expenses)))
    (is (= 3763 (total-amount (category-is "books") clj-expenses))))

(def java-expenses [(Expense. "2009-8-24" 44 95 "books" "amazon.com")
                    (Expense. "2009-8-25" 29 11 "gas" "shell")])

(deftest test-java-expenses-total
    (let [total-cents (map #(.amountInCents %) java-expenses)]
        (is (= 7406 (apply + total-cents)))))

(def mixed-expenses (concat clj-expenses java-expenses))

(deftest test-mixed-expenses-total
    (is (= 49983 (total-amount mixed-expenses)))
    (is (= 8258 (total-amount (category-is "books") mixed-expenses))))

(use 'clojure.test) (run-tests 'clj-in-act.ch9.expense-test)

Querying modus operandi

In [ ]:
; A function that discerns what data types implement a particular modus operandi

(defn implementors [modus-operandi method]
    (get-in modus-operandi [method :implementors]))

(implementors ExpenseCalculations :is-category?)


In [ ]:
; A function that when given a class of a particular data type 
; can tell you if it implements a particular method of a modus operandi

(defn implements? [implementor modus-operandi method]
    (some #{implementor} (implementors modus-operandi method)))

; Test with a class that implements 'is-category?'

(implements? com.curry.expenses.Expense ExpenseCalculations :is-category?)


In [ ]:
; Test with a class that doesn't implement 'is-category?'

(implements? java.util.Date ExpenseCalculations :is-category?)

In [ ]:
; A function to see if a class implements a modus operandi completely

(defn full-implementor? [implementor modus-operandi]
    (->> (keys modus-operandi)
         (map #(implements? implementor modus-operandi %))
         (not-any? nil?)))

; A class that fully implements modus operandi

(full-implementor? com.curry.expenses.Expense ExpenseCalculations)


In [ ]:
; A class that partially implements modus operandi

(detail-modus-operandi ExpenseCalculations 
    java.util.Date
    (total-cents [e] (rand-int 1000)))

(full-implementor? java.util.Date ExpenseCalculations)

## Examining the data types side of the expression problem with protocols

In [ ]:
; Third version of the app, implemented with Clojure protocols

(ns clj-in-act.ch9.expense-protocol
    (:import [java.text SimpleDateFormat]
             [java.util Calendar]))

(defn new-expense [date-string dollars cents category merchant-name]
    (let [calendar-date (Calendar/getInstance)]
        (.setTime calendar-date (.parse (SimpleDateFormat. "yyyy-MM-dd") date-string))
        {:date calendar-date
         :amount-dollars dollars
         :amount-cents cents
         :category category
         :merchant-name merchant-name}))

(defprotocol ExpenseCalculations
    (total-cents [e])
    (is-category? [e category]))

(extend-protocol ExpenseCalculations
    clojure.lang.IPersistentMap
    (total-cents [e]
        (-> (:amount-dollars e)
            (* 100)
            (+ (:amount-cents e))))
    (is-category? [e some-category]
        (= (:category e) some-category)))

(extend-protocol ExpenseCalculations
    com.curry.expenses.Expense
    (total-cents [e]
        (.amountInCents e))
    (is-category? [e some-category]
        (= (.getCategory e) some-category)))

(defn category-is [category]
    #(is-category? % category))

(defn total-amount
    ([expenses-list]
        (total-amount (constantly true) expenses-list))
    ([pred expenses-list]
        (->> expenses-list
             (filter pred)
             (map total-cents)
             (apply +))))


In [ ]:
; General form of the 'defprotocol' syntax

(defprotocol AProtocolName
    "A doc string for AProtocol abstraction"
    (bar [this a b] "bar docs") 
    (baz [this a] [this a b] [this a b c] "baz docs"))


In [ ]:
; The 'extend-protocol' macro is defined on top of 
; another macro 'extend-type'

(extend-type com.curry.expenses.Expense
    ExpenseCalculations
    (total-cents [e]
        (.amountInCents e))
    (is-category? [e some-category]
        (= (.getCategory e) some-category)))


In [ ]:
; The 'extend' function does the work of registering protocol participants 
; and associating the methods with the right data types. 

(extend com.curry.expenses.Expense
    ExpenseCalculations {
        :total-cents (fn [e] (.amountInCents e))
        :is-category? (fn [e some-category] (= (.getCategory e) some-category))})


In [ ]:
;  Protocols can be extended on nil

(extend-protocol ExpenseCalculations nil
    (total-cents [e] 0))


Reflecting on protocols

In [ ]:
; The 'extends?'  function can be used to check 
; if a particular data type participates in a protocol

(extends? ExpenseCalculations com.curry.expenses.Expense)

(extends? ExpenseCalculations clojure.lang.IPersistentMap)

(extends? ExpenseCalculations java.util.Date)

In [ ]:
; The 'extenders' function lists all the data types 
; that participate in a protocol.

(extenders ExpenseCalculations)

In [ ]:
; The 'satisfies?'  function can be used to check 
; if a particular instance of a data type participates in a protocol

(satisfies? ExpenseCalculations 
    (com.curry.expenses.Expense. "10-10-2010" 20 95 "books" "amzn"))

(satisfies? ExpenseCalculations 
    (new-expense "10-10-2010" 20 95 "books" "amzn"))

(satisfies? ExpenseCalculations (java.util.Random.))

### Defining data types with deftype, defrecord, and reify

defrecord

In [ ]:
; A simple record example

(defrecord NewExpense 
    [date amount-dollars amount-cents category merchant-name])


In [ ]:
; Importing a class with munged Java classpath (underscores instead of hyphens)

(import 'chapter_protocols.expense_record.NewExpense) 

; Using a Java constructor on the class

(NewExpense. "2010-04-01" 29 95 "gift" "1-800-flowers")


In [ ]:
; Using constructor functions that are imported in namespace (more idiomatic)

(require '[clj-in-act.ch9.expense-record :as er]) 

;  ->RECORDNAME constructor accepts positional parameters.

(er/->NewExpense "2010-04-01" 29 95 "gift" "1-800-flowers") 

; map->RECORDNAME constructor accepts  a single map.

(er/map->NewExpense 
    {:date "2010-04-01", :merchant-name "1-800-flowers", :message "April fools!"})

In [ ]:
; Fourth version of the app, implemented with Clojure protocols and records

(ns clj-in-act.ch9.expense-record
    (:import [java.text SimpleDateFormat]
             [java.util Calendar]))

(defrecord NewExpense 
    [date amount-dollars amount-cents category merchant-name])

(defn new-expense [date-string dollars cents category merchant-name]
    (let [calendar-date (Calendar/getInstance)]
    (.setTime calendar-date (.parse (SimpleDateFormat. "yyyy-MM-dd")  date-string))
    (->NewExpense calendar-date dollars cents category merchant-name)))

(defprotocol ExpenseCalculations
    (total-cents [e])
    (is-category? [e category]))

(extend-type NewExpense
    ExpenseCalculations
    (total-cents [e]
        (-> (:amount-dollars e)
            (* 100)
            (+ (:amount-cents e))))
    (is-category? [e some-category]
        (= (:category e) some-category)))

(extend com.curry.expenses.Expense
    ExpenseCalculations {
        :total-cents (fn [e] (.amountInCents e))
        :is-category? (fn [e some-category] (= (.getCategory e) some-category))})

(extend-protocol ExpenseCalculations nil
    (total-cents [e] 0))

(defn category-is [category]
    #(is-category? % category))

(defn total-amount
    ([expenses-list]
        (total-amount (constantly true) expenses-list))
    ([pred expenses-list]
        (->> expenses-list
             (filter pred)
             (map total-cents)
             (apply +))))


Examples to illustrate records’ maplike features

In [ ]:
; Defining a record 

(defrecord Foo [a b])


In [ ]:
; Instantiating a record 

(def foo (->Foo 1 2))

In [ ]:
; Associating a new pair returns a new record with key added.

(assoc foo :extra-key 3) 


In [ ]:
; Dissociating a new pair returns a new record with key removed.

(dissoc (assoc foo :extra-key 3) :extra-key)


In [ ]:
; Dissociating a field key return an ordinary map.

(dissoc foo :a) 


In [ ]:
; But records aren’t callable like maps.

(foo :a) 

In [ ]:
; Fifth version of the app, with Clojure records 
; and their inline implementation of protocols 

(ns clj-in-act.ch9.expense-record-2
    (:import [java.text SimpleDateFormat]
             [java.util Calendar]))

(defprotocol ExpenseCalculations
    (total-cents [e])
    (is-category? [e category]))

(defrecord NewExpense [date amount-dollars amount-cents category merchant-name]
    ExpenseCalculations
    (total-cents [this]
        (-> amount-dollars
            (* 100)
            (+ amount-cents)))
    (is-category? [this some-category]
        (= category some-category)))

(defn new-expense [date-string dollars cents category merchant-name]
    (let [calendar-date (Calendar/getInstance)]
        (.setTime calendar-date (.parse (SimpleDateFormat. "yyyy-MM-dd") date-string))
        (->NewExpense calendar-date dollars cents category merchant-name)))

(extend com.curry.expenses.Expense
    ExpenseCalculations {
        :total-cents (fn [e] (.amountInCents e))
        :is-category? (fn [e some-category] (= (.getCategory e) some-category))})

(extend-protocol ExpenseCalculations nil
    (total-cents [e] 0))

(defn category-is [category]
    #(is-category? % category))

(defn total-amount
    ([expenses-list]
        (total-amount (constantly true) expenses-list))
    ([pred expenses-list]
        (->> expenses-list
             (filter pred)
             (map total-cents)
             (apply +))))


deftype

In [ ]:
; The 'deftype' creates a bare Java class, without the added properties of records

(deftype Mytype [a b])


reify

In [ ]:
; The reify macro takes a protocol (an abstract set of methods)
; and creates an instance of an anonymous data type that implements that protocol

(defn new-expense [date-string dollars cents 
                   category merchant-name]
    (let [calendar-date (Calendar/getInstance)]
        (.setTime calendar-date (.parse (SimpleDateFormat. "yyyy-MM-dd") date-string))
        (reify ExpenseCalculations
            (total-cents [this]
                (-> dollars
                    (* 100)
                    (+ cents)))
                (is-category? [this some-category]
                    (= category some-category))))) ; Returns a closure
